## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

from torchvision import transforms
from torchvision.utils import make_grid

import math
import random

from PIL import Image, ImageOps, ImageEnhance
import numbers

import matplotlib.pyplot as plt
%matplotlib inline

colStart=2

## Explore the Data

In [2]:
# 28x28 pixels image, has first column label
train_df = pd.read_csv('./train.csv', sep=',')

n_train = len(train_df)
n_pixels = len(train_df.columns) - 1
n_class = len(set(train_df['label']))

print("Number of training samples:",n_train) #{0}'.format(n_train))
print('Number of training pixels: ', n_pixels)
print('Number of classes: ', n_class)

Number of training samples: 42000
Number of training pixels:  784
Number of classes:  10


In [3]:
# 28x28 pixels image, has first column label
test_df = pd.read_csv('./test.csv', sep=',')

n_test = len(test_df)
n_pixels = len(test_df.columns) - 1
n_class = len(set(test_df['label']))

print("Number of training samples:",n_test)
print('Number of training pixels: ', n_pixels)
print('Number of classes: ', n_class)

KeyError: 'label'

## Display Some Images

In [ ]:
random_sel = np.random.randint(n_train, size=14)

def dispImage(df, row, colStart, nrow):
    grid = make_grid(torch.Tensor((df.iloc[row, colStart:].to_numpy()/255.).reshape((-1, 28, 28))).unsqueeze(1), nrow=nrow)
    plt.rcParams['figure.figsize'] = (32, 150)
    plt.imshow(grid.numpy().transpose((1,2,0)))
    plt.axis('off')

dispImage(train_df, random_sel, 2, 7)
    
print(*list(train_df.iloc[random_sel, 1].values), sep = ', ')

In [ ]:
def showHistogram(df):
    plt.rcParams['figure.figsize'] = (8, 5)
    plt.bar(df['label'].value_counts().index, df['label'].value_counts())
    plt.xticks(np.arange(n_class))
    plt.xlabel('Class', fontsize=16)
    plt.ylabel('Count', fontsize=16)
    plt.grid('on', axis='y')

showHistogram(train_df)
showHistogram(test_df)

print("Shape of the train_df is: ", train_df.shape)
print("Shape of the test_df is: ", test_df.shape)

## Data Loader Class

In [ ]:
class MNIST_data(Dataset):
    """MNIST dtaa set"""
    
    def __init__(self, 
                 file_path, 
                 isTest = bool,
                 transform = transforms.Compose([transforms.ToPILImage(), 
                                                 transforms.ToTensor(), 
                                                 transforms.Normalize(mean=(0.5,), std=(0.5,))
                                                ])
                ):
        
        df = pd.read_csv(file_path, sep=",")
        
        if isTest:
            # test data
            # X is pixel values
            # Y is labels
            self.X = df.iloc[:,1:].values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = None
        else:
            # training data
            self.X = df.iloc[:,1:].values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = torch.from_numpy(df.iloc[:,1].values)
         
        self.transform = transform
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])
        

    def getY(self, idx):
        if self.y is not None:
            return self.y[idx]
        else:
            print("There is no Y component")

## Random Rotation Transformation
Randomly rotate the image. Available in upcoming torchvision but not now.

In [ ]:
class RandomRotation(object):
    """
    https://github.com/pytorch/vision/tree/master/torchvision/transforms
    Rotate the image by angle.
    Args:
        degrees (sequence or float or int): Range of degrees to select from.
            If degrees is a number instead of sequence like (min, max), the range of degrees
            will be (-degrees, +degrees).
        resample ({PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC}, optional):
            An optional resampling filter.
            See http://pillow.readthedocs.io/en/3.4.x/handbook/concepts.html#filters
            If omitted, or if the image has mode "1" or "P", it is set to PIL.Image.NEAREST.
        expand (bool, optional): Optional expansion flag.
            If true, expands the output to make it large enough to hold the entire rotated image.
            If false or omitted, make the output image the same size as the input image.
            Note that the expand flag assumes rotation around the center and no translation.
        center (2-tuple, optional): Optional center of rotation.
            Origin is the upper left corner.
            Default is the center of the image.
    """

    def __init__(self, degrees, resample=False, expand=False, center=None):
        if isinstance(degrees, numbers.Number):
            if degrees < 0:
                raise ValueError("If degrees is a single number, it must be positive.")
            self.degrees = (-degrees, degrees)
        else:
            if len(degrees) != 2:
                raise ValueError("If degrees is a sequence, it must be of len 2.")
            self.degrees = degrees

        self.resample = resample
        self.expand = expand
        self.center = center

    @staticmethod
    def get_params(degrees):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        angle = np.random.uniform(degrees[0], degrees[1])

        return angle

    def __call__(self, img):
        """
            img (PIL Image): Image to be rotated.
        Returns:
            PIL Image: Rotated image.
        """
        
        def rotate(img, angle, resample=False, expand=False, center=None):
            """Rotate the image by angle and then (optionally) translate it by (n_columns, n_rows)
            Args:
            img (PIL Image): PIL Image to be rotated.
            angle ({float, int}): In degrees degrees counter clockwise order.
            resample ({PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC}, optional):
            An optional resampling filter.
            See http://pillow.readthedocs.io/en/3.4.x/handbook/concepts.html#filters
            If omitted, or if the image has mode "1" or "P", it is set to PIL.Image.NEAREST.
            expand (bool, optional): Optional expansion flag.
            If true, expands the output image to make it large enough to hold the entire rotated image.
            If false or omitted, make the output image the same size as the input image.
            Note that the expand flag assumes rotation around the center and no translation.
            center (2-tuple, optional): Optional center of rotation.
            Origin is the upper left corner.
            Default is the center of the image.
            """
                
            return img.rotate(angle, resample, expand, center)

        angle = self.get_params(self.degrees)

        return rotate(img, angle, self.resample, self.expand, self.center)

## Random Vertical and Horizontal Shift

In [ ]:
class RandomShift(object):
    def __init__(self, shift):
        self.shift = shift
        
    @staticmethod
    def get_params(shift):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        hshift, vshift = np.random.uniform(-shift, shift, size=2)

        return hshift, vshift 
    def __call__(self, img):
        hshift, vshift = self.get_params(self.shift)
        
        return img.transform(img.size, Image.AFFINE, (1,0,hshift,0,1,vshift), resample=Image.BICUBIC, fill=1)
    

## Load the Data into Tensors
For the training set, apply random rotation within the range of (-45, 45) degrees, shift by (-3, 3) pixels and normalize pixel values to [-1, 1]. For the test set, only apply nomalization.

In [ ]:
batch_size = 64

train_dataset = MNIST_data('98_train.csv', isTest=False, transform = transforms.Compose(
                            [transforms.ToPILImage(), RandomRotation(degrees=20), RandomShift(3),
                             transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))]))
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


train_x_dataset = MNIST_data('98_train.csv', isTest=True)
train_x_loader = torch.utils.data.DataLoader(dataset=train_x_dataset, batch_size=batch_size, shuffle=False)


test_dataset = MNIST_data('98_test.csv', isTest=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


test_y_dataset = MNIST_data('98_test.csv', isTest=False)
test_y_loader = torch.utils.data.DataLoader(dataset=test_y_dataset, batch_size=batch_size, shuffle=False)

## Demonstration of Transformations 

In [ ]:
rotate = RandomRotation(60)
shift = RandomShift(5)
composed = transforms.Compose([RandomRotation(20),
                               RandomShift(3)])

# Apply each of the above transforms on sample.
fig = plt.figure()
sample = transforms.ToPILImage()(train_df.iloc[65,1:].values.reshape((28,28)).astype(np.uint8)[:,:,None])
for i, tsfrm in enumerate([rotate, shift, composed]):
    transformed_sample = tsfrm(sample)

    ax = plt.subplot(1, 3, i + 1)
    plt.tight_layout()
    ax.set_title(type(tsfrm).__name__)
    ax.imshow(np.reshape(np.array(list(transformed_sample.getdata())), (-1,28)), cmap='gray')    

plt.show()

## Network Structure

In [ ]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
          
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
          
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(64 * 7 * 7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            
            nn.Dropout(p = 0.5),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            
            nn.Dropout(p = 0.5),
            nn.Linear(512, 10),
        )
          
        for m in self.features.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
        for m in self.classifier.children():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform(m.weight)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        
        return x     

## Initialize the Network Model
Initialize the network as 'model'

In [ ]:
model = Net()

optimizer = optim.Adam(model.parameters(), lr=0.003)

criterion = nn.CrossEntropyLoss()

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

## Training and Evaluation

In [ ]:
def train(epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        exp_lr_scheduler.step()
        
        if (batch_idx + 1)% 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.data))

In [ ]:
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    for data, target in data_loader:
        data, target = Variable(data), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += F.cross_entropy(output, target, size_average=False).data

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

In [ ]:
n_epochs = 1

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

## Prediction

In [ ]:
def prediciton(data_loader):
    model.eval()
    test_pred = torch.LongTensor()
    
    for i, data in enumerate(data_loader):
        data = Variable(data)
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = model(data)
        #print(output)
        
        pred = output.cpu().data.max(1, keepdim=True)[1]
        #print(pred)
        test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred

In [ ]:
train_pred = prediciton(train_x_loader)
train_prediction_df = pd.DataFrame(np.c_[np.arange(1, len(train_dataset)+1)[:,None], train_pred.numpy()], 
                      columns=['ImageId', 'label'])
print(len(train_pred))

In [ ]:
train_predictions = train_prediction_df.iloc[:, 1].to_numpy()

count = 0
train_index = np.array(range(300))

for i in range(len(train_predictions)):
    if train_dataset.getY(i).item() == train_predictions[i]:
        continue
    else:
        train_index[count] = i
        count +=1
        print("Label is: ", train_dataset.getY(i).item(), "Predcition is: ", train_predictions[i], "index is: ", i)
print(count)

In [ ]:
# Confusion matrix whose i-th row and j-th column entry indicates the number of samples with 
# true label being i-th class and predicted label being j-th class.
con_train = confusion_matrix(train_dataset.y, train_predictions)

# Class 0 is treated as Negative
tn, fp, fn, tp = con_train.ravel()

prec_train = tp/(tp+fp)
rec_train = tp/(tp+fn)

print("TP: ", tp, "FP: ", fp)
print("FN: ", fn, "  TN: ", tn)

print("Precision is: ", prec_train)
print("Recall is: ", rec_train)

print(con_train)

In [ ]:
num = np.arange(count)

print("  Label -- Pred -- Index")

count2 = 0
for i in range(count):
    count2 += 1
    print(count2, "   ",train_df.iloc[train_index[num[i]],0], "  ", 
          train_predictions[train_index[num[i]]], "     ", train_index[num[i]], 
         )


dispImage(train_df, train_index[:count], 2, 6)


## Performance of Test Dataset

In [ ]:
test_pred = prediciton(test_loader)
test_prediction_df = pd.DataFrame(np.c_[np.arange(1, len(test_dataset)+1)[:,None], test_pred.numpy()], 
                      columns=['ImageId', 'Label'])
print(len(test_pred))

In [ ]:
test_predictions = test_prediction_df.iloc[:, 1].to_numpy()
test_labels = test_df.iloc[:,0].to_numpy()
print(len(test_predictions))

In [ ]:
count3 = 0
test_index = np.array(range(300))

for i in range(len(test_predictions)):
    if test_y_dataset.getY(i).item() == test_predictions[i]:
        continue
    else:
        test_index[count3] = i
        count3 += 1
        print("Label is: ", test_y_dataset.getY(i).item(), "Predcition is: ", 
              test_predictions[i], "index is: ", i)
print("Count3: ", count3)

In [ ]:
con_test = confusion_matrix(test_y_dataset.y, test_predictions)
# Confusion matrix whose i-th row and j-th column entry indicates the number of samples with 
# true label being i-th class and predicted label being j-th class.

# Class 0 is treated as Negative
tn, fp, fn, tp = con_test.ravel()

prec_test = tp/(tp+fp)
rec_test = tp/(tp+fn)

print("TP: ", tp, "FP: ", fp)
print("FN: ", fn, "  TN: ", tn)

print("Precision is: ", prec_test)
print("Recall is: ", rec_test)

print(con_test)

In [ ]:
num = np.arange(count3)

print("  Label -- Pred -- Index")

count4 = 0
for i in range(count3):
    count4 += 1
    print(count4, "   ",test_df.iloc[test_index[num[i]],0], "  ", 
          test_predictions[test_index[num[i]]], "     ", test_index[num[i]], 
         )


dispImage(test_df, test_index[:count3], 2, 6)